In [1]:
import numpy as np
import inspect
import matplotlib.pyplot as plt
from tkinter import filedialog
from tkinter import *
from roipoly import RoiPoly

import openpiv.tools as tools
import openpiv.pyprocess
import openpiv.scaling
import openpiv.validation
import openpiv.filters #NaN filters

In [2]:
%matplotlib qt
%run -i testing

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def im2double(im):
    info = np.iinfo(im.dtype) # Get the data type of the input image
    return im.astype(np.float) / info.max # Divide all values by the largest possible value in the datatype

root = Tk()


#root.directory = filedialog.askdirectory()

root.pc_name = filedialog.askopenfilename(initialdir = '.',
                                          title = "Select phase contrast img",
                                          filetypes = (("tiff files","*.tiff"),
                                          ("all files","*.*")))

root.load_name = filedialog.askopenfilename(initialdir = '.',
                                     title = "Select loadimg",
                                     filetypes = (("tiff files","*.tif"),
                                                  ("all files","*.*")))
root.nulf_name = filedialog.askopenfilename(initialdir = '.',
                                     title = "Select nulfimg",
                                     filetypes = (("tiff files","*.tif"),("all files","*.*")))


frame_c = tools.imread(root.pc_name)
frame_a = tools.imread(root.nulf_name)
frame_b = tools.imread(root.load_name)

root.withdraw()

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


''

In [4]:
%matplotlib qt
fig = plt.figure()
plt.imshow(frame_c.astype(np.int32),CMAP='gray',vmin=0,vmax=0.5*frame_c.astype(np.int32).max(),origin='lower')
plt.title("left click: line segment; right click or double click: close region")
plt.show(block=False)

cell_roi = RoiPoly(color='r',fig=fig,show_fig=False)

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-4-2a21a6aebdf6>:3: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  plt.imshow(frame_c.astype(np.int32),CMAP='gray',vmin=0,vmax=0.5*frame_c.astype(np.int32).max(),origin='lower')


In [7]:
%matplotlib qt
fig = plt.figure()
plt.imshow(frame_c.astype(np.int32),CMAP='gray',vmin=0,vmax=0.5*frame_c.astype(np.int32).max(),origin='lower')
cell_roi.display_roi()
plt.title('With celltrace')
plt.show(block=False)

#plt.figure()
#plt.plot(cell_roi.x,cell_roi.y,color='r',linewidth=2.5)
#plt.plot(cell_roi.x + [cell_roi.x[0]],cell_roi.y + [cell_roi.y[0]],color='b')
#plt.show(block=False)
#cell_roi does not close boundary itself
cellx = np.array(cell_roi.x + [cell_roi.x[0]])
celly = np.array(cell_roi.y + [cell_roi.y[0]])


/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-7-d022dfc0da2c>:3: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  plt.imshow(frame_c.astype(np.int32),CMAP='gray',vmin=0,vmax=0.5*frame_c.astype(np.int32).max(),origin='lower')


In [5]:
cimg = np.zeros([frame_a.shape[0],frame_a.shape[1],3],dtype=np.uint8)
cimg[:,:,0] = frame_a
cimg[:,:,1] = frame_b
plt.figure()
plt.imshow(cimg,vmin=0,vmax=0.01*cimg[:,:,0].max(),origin='lower')

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
u, v , sig2noise= openpiv.pyprocess.extended_search_area_piv( frame_a.astype(np.int32), frame_b.astype(np.int32), window_size=32, overlap=16, dt=1, search_area_size=48, sig2noise_method='peak2peak' )

xgrid, ygrid = openpiv.pyprocess.get_coordinates( image_size=frame_a.shape, search_area_size=48, overlap=16 )
#xgrid, ygrid = openpiv.pyprocess.get_coordinates( image_size=frame_a.shape, window_size=32, overlap=16 )
#openpiv changed their method with the default extend_search_area == window_size, hence the parameter in get_coordinates has changed to
# extended search area. Please check the parameters in extended_search_area_piv

u, v, mask = openpiv.validation.sig2noise_val( u, v, sig2noise, threshold = 2.8 )
u, v = openpiv.filters.replace_outliers( u, v, method='localmean', max_iter=10, tol=1e-4, kernel_size=2 )
#x, y, u, v = openpiv.scaling.uniform(x, y, u, v, scaling_factor = 96.52 )

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
plt.figure()
#plt.imshow(frame_a.astype(np.int32),cmap='gray',vmin=0,vmax=0.1*frame_a.astype(np.int32).max())
plt.quiver(xgrid,ygrid,u,v,color='c')
#plt.plot(celly,cellx,color='r')
plt.show()
u

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
  

In [24]:
#cellTrace = np.c_[cellx,celly]
#np.array_equal(cellTrace[:,0],cellx)
np.savez('testcell',xgrid=xgrid,ygrid=ygrid,xdisp=u,ydisp=v,cellTrace=np.c_[cellx,celly])

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
#load npz format:
npzfile = np.load('testcell.npz')
# print all npzfile field: npzfile.files
# access a field in npzfile:
# npzfile['cellTrace']

/home/unstable/miniconda3/envs/pyTFM/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [77]:
print(frame_a.shape,frame_b.shape,frame_c.shape)

(801, 969) (801, 969) (1040, 1392)


NameError: name 'uint16' is not defined

In [2]:
import sys
for p in sys.path:
    print(p)

P:\#Code\pyadaptation\testopenPIV
D:\ProgramData\Miniconda3\python37.zip
D:\ProgramData\Miniconda3\DLLs
D:\ProgramData\Miniconda3\lib
D:\ProgramData\Miniconda3

D:\ProgramData\Miniconda3\lib\site-packages
D:\ProgramData\Miniconda3\lib\site-packages\win32
D:\ProgramData\Miniconda3\lib\site-packages\win32\lib
D:\ProgramData\Miniconda3\lib\site-packages\Pythonwin
D:\ProgramData\Miniconda3\lib\site-packages\IPython\extensions
C:\Users\surface\.ipython


In [9]:
import cv2
im = cv2.imread("a")
cv2.imshow("Image",im)

error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\highgui\src\window.cpp:356: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
